##### Copyright 2018 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Save and restore models

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_restore_models"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_restore_models.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_restore_models.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Model progress can be saved during—and after—training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

* code to create the model, and
* the trained weights, or parameters, for the model

Sharing this data helps others understand how the model works and try it themselves with new data.

Caution: Be careful with untrusted code—TensorFlow models are code. See [Using TensorFlow Securely](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) for details.

### Options

There are different ways to save TensorFlow models—depending on the API you're using. This guide uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow. For other approaches, see the TensorFlow  [Save and Restore](https://www.tensorflow.org/guide/saved_model) guide or [Saving in eager](https://www.tensorflow.org/guide/eager#object-based_saving).


## Setup

### Installs and imports

Install and import TensorFlow and dependencies:

In [3]:
!pip install h5py pyyaml
!pip install tf_nightly

### Get an example dataset

We'll use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to train our model to demonstrate saving weights. To speed up these demonstration runs, only use the first 1000 examples:

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.15.0-dev20190626'

In [5]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Define a model

Let's build a simple model we'll use to demonstrate saving and loading weights.

In [6]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model


# Create a basic model instance
model = create_model()
model.summary()

W0629 12:11:42.715696 4583224768 deprecation.py:506] From /anaconda3/envs/TestEnv/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1624: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

The primary use case is to automatically save checkpoints *during* and at *the end* of training. This way you can use a trained model without having to retrain it, or pick-up training where you left of—in case the training process was interrupted.

`tf.keras.callbacks.ModelCheckpoint` is a callback that performs this task. The callback takes a couple of arguments to configure checkpointing.

### Checkpoint callback usage

Train the model and pass it the `ModelCheckpoint` callback:

In [7]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

W0629 12:11:48.169029 4583224768 deprecation.py:323] From /anaconda3/envs/TestEnv/lib/python3.7/site-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:460: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 576/1000 [================>.............] - ETA: 0s - loss: 1.4785 - acc: 0.5556
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 428us/sample - loss: 1.1518 - acc: 0.6590 - val_loss: 0.7047 - val_acc: 0.7890
Epoch 2/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.4262 - acc: 0.8906
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 170us/sample - loss: 0.4119 - acc: 0.8820 - val_loss: 0.5577 - val_acc: 0.8300
Epoch 3/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.2665 - acc: 0.9392
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 153us/sample - loss: 0.2768 - acc: 0.9330 - val_loss: 0.4948 - val_acc: 0.8490
Epoch 4/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.1998 - acc: 0.9531
Epoch 00004: saving model to training_1/cp.ckpt
1000/10

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [8]:
!ls {checkpoint_dir}

checkpoint                  cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002 cp.ckpt.index


Create a new, untrained model. When restoring a model from only weights, you must have a model with the same architecture as the original model. Since it's the same model architecture, we can share weights despite that it's a different *instance* of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [9]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 82us/sample - loss: 2.3065 - acc: 0.0960
Untrained model, accuracy:  9.60%


Then load the weights from the checkpoint, and re-evaluate:

In [10]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 34us/sample - loss: 0.4027 - acc: 0.8730
Restored model, accuracy: 87.30%


### Checkpoint callback options

The callback provides several options to give the resulting checkpoints unique names, and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every 5-epochs:


In [11]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

W0629 12:12:38.067486 4583224768 callbacks.py:861] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


Now, look at the resulting checkpoints and choose the latest one:

In [12]:
! ls {checkpoint_dir}

checkpoint                       cp-0025.ckpt.data-00001-of-00002
cp-0000.ckpt.data-00000-of-00002 cp-0025.ckpt.index
cp-0000.ckpt.data-00001-of-00002 cp-0030.ckpt.data-00000-of-00002
cp-0000.ckpt.index               cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00000-of-00002 cp-0030.ckpt.index
cp-0005.ckpt.data-00001-of-00002 cp-0035.ckpt.data-00000-of-00002
cp-0005.ckpt.index               cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00000-of-00002 cp-0035.ckpt.index
cp-0010.ckpt.data-00001-of-00002 cp-0040.ckpt.data-00000-of-00002
cp-0010.ckpt.index               cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00000-of-00002 cp-0040.ckpt.index
cp-0015.ckpt.data-00001-of-00002 cp-0045.ckpt.data-00000-of-00002
cp-0015.ckpt.index               cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00000-of-00002 cp-0045.ckpt.index
cp-0020.ckpt.data-00001-of-00002 cp-0050.ckpt.data-00000-of-00002
cp-0020.ckpt.index               cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.d

In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

Note: the default tensorflow format only saves the 5 most recent checkpoints.

To test, reset the model and load the latest checkpoint:

In [14]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 153us/sample - loss: 0.4841 - acc: 0.8790
Restored model, accuracy: 87.90%


## What are these files?

The above code stores the weights to a collection of [checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)-formatted files that contain only the trained weights in a binary format. Checkpoints contain:
* One or more shards that contain your model's weights.
* An index file that indicates which weights are stored in a which shard.

If you are only training a model on a single machine, you'll have one shard with the suffix: `.data-00000-of-00001`

## Manually save weights

Above you saw how to load the weights into a model.

Manually saving the weights is just as simple, use the `Model.save_weights` method.

In [15]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

W0629 12:13:01.971766 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.iter
W0629 12:13:01.972497 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.beta_1
W0629 12:13:01.973164 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.beta_2
W0629 12:13:01.973649 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.decay
W0629 12:13:01.974223 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0629 12:13:01.974640 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.kernel
W0629 12:13:01.976048 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.bias
W0629 12:13:01.976592 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.kernel
W0629 12:13:01.977646 4583224768 util.py:2

1000/1000 [==============================] - 0s 190us/sample - loss: 0.4841 - acc: 0.8790
Restored model, accuracy: 87.90%


## Save the entire model

The entire model can be saved to a file that contains the weight values, the model's configuration, and even the optimizer's configuration (depends on set up). This allows you to checkpoint a model and resume training later—from the exact same state—without access to the original code.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([HDF5](https://js.tensorflow.org/tutorials/import-keras.html), [Saved Model](https://js.tensorflow.org/tutorials/import-saved-model.html)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([HDF5](https://www.tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_), [Saved Model](https://www.tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_))

### As an HDF5 file

Keras provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. For our purposes, the saved model can be treated as a single binary blob.

In [16]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 329us/sample - loss: 1.1759 - acc: 0.6560
Epoch 2/5
1000/1000 [==============================] - 0s 95us/sample - loss: 0.4257 - acc: 0.8890
Epoch 3/5
1000/1000 [==============================] - 0s 96us/sample - loss: 0.2846 - acc: 0.9190
Epoch 4/5
1000/1000 [==============================] - 0s 101us/sample - loss: 0.2079 - acc: 0.9510
Epoch 5/5
1000/1000 [==============================] - 0s 99us/sample - loss: 0.1448 - acc: 0.9670


Now recreate the model from that file:

In [17]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

W0629 12:13:12.660977 4583224768 deprecation.py:506] From /anaconda3/envs/TestEnv/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0629 12:13:12.662508 4583224768 deprecation.py:506] From /anaconda3/envs/TestEnv/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Check its accuracy:

In [18]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 223us/sample - loss: 0.4310 - acc: 0.8650
Restored model, accuracy: 86.50%


This technique saves everything:

* The weight values
* The model's configuration(architecture)
* The optimizer configuration

Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from `tf.train`). When using those you will need to re-compile the model after loading, and you will lose the state of the optimizer.


### As a `saved_model`

Caution: This method of saving a `tf.keras` model is experimental and may change in future versions.

Build a fresh model:

In [19]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 374us/sample - loss: 1.0788 - acc: 0.6880
Epoch 2/5
1000/1000 [==============================] - 0s 98us/sample - loss: 0.4219 - acc: 0.8940
Epoch 3/5
1000/1000 [==============================] - 0s 99us/sample - loss: 0.2955 - acc: 0.9210
Epoch 4/5
1000/1000 [==============================] - 0s 97us/sample - loss: 0.1976 - acc: 0.9490
Epoch 5/5
1000/1000 [==============================] - 0s 103us/sample - loss: 0.1543 - acc: 0.9640


Create a `saved_model`:

In [20]:
import time

saved_model_path = "./saved_models/"+str(int(time.time()))
tf.contrib.saved_model.save_keras_model(model, saved_model_path)

W0629 12:13:33.368090 4583224768 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0629 12:13:33.599852 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.iter
W0629 12:13:33.600113 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.beta_1
W0629 12:13:33.600183 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.beta_2
W0629 12:13:33.600239 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.decay
W0629 12:13:33.600289 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0629 12:13:33.600337 4583224768 util.py:252] Unresolved object 

Have a look in the directory:

In [21]:
!ls {saved_model_path}

assets         saved_model.pb variables


Reload a fresh keras model from the saved model.

In [22]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.summary()

W0629 12:13:57.872083 4583224768 deprecation.py:323] From <ipython-input-22-f5ba023fc0b4>:1: load_from_saved_model (from tensorflow.python.keras.saving.saved_model_experimental) is deprecated and will be removed in a future version.
Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.
W0629 12:13:58.026103 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.beta_1
W0629 12:13:58.026704 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.beta_2
W0629 12:13:58.027164 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.decay
W0629 12:13:58.027686 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0629 12:13:58.028268 4583224768 util.py:252] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.kernel
W0629 12:13:58.028668 4583224768 util.py:252] Unresolved object 

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Run the restored model.

In [23]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 267us/sample - loss: 0.4415 - acc: 0.8590
Restored model, accuracy: 85.90%


## What's Next

That was a quick guide to saving and loading in with `tf.keras`.

* The [tf.keras guide](https://www.tensorflow.org/guide/keras) shows more about saving and loading models with `tf.keras`.

* See [Saving in eager](https://www.tensorflow.org/guide/eager#object_based_saving) for saving during eager execution.

* The [Save and Restore](https://www.tensorflow.org/guide/saved_model) guide has low-level details about TensorFlow saving.